In [1]:
import sys
import pandas as pd
import scipy

In [2]:
# эта функция нам поможет считывать данные
from os import path

data_dir = "/home/ubuntu/data/movielens/ml-latest-small"
def read_csv(filename: str) -> pd.DataFrame:
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

ratings = read_csv("ratings")
movies = read_csv("movies")

In [3]:
# пользователи ставят фильмам оценки
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
# про фильмы знаем названия и жанры
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
print("максимальный ID фильма")
last_movie_id = ratings["movieId"].max()
print(last_movie_id)
print("всего фильмов")
print(len(ratings["movieId"].unique()))
print("максимальный ID пользователя")
last_user_id = ratings["userId"].max()
print(last_user_id)
print("всего пользователей")
print(len(ratings["userId"].unique()))

максимальный ID фильма
163949
всего фильмов
9066
максимальный ID пользователя
671
всего пользователей
671


In [6]:
ratings["movie_id"] = ratings["movieId"].astype("category").cat.codes.copy() + 1

In [7]:
print("максимальный ID фильма")
last_movie_id = ratings["movie_id"].max()
print(last_movie_id)
print("всего фильмов")
print(len(ratings["movie_id"].unique()))

максимальный ID фильма
9066
всего фильмов
9066


In [8]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [9]:
user_x_item = ratings[["userId", "movie_id"]].as_matrix()
user_x_item

array([[   1,   31],
       [   1,  834],
       [   1,  860],
       ..., 
       [ 671, 4598],
       [ 671, 4611],
       [ 671, 4697]])

In [10]:
user_item_matrix = csr_matrix(
    (
        [1] * len(user_x_item),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1)
)

sparse_info(user_item_matrix)

Размерности матрицы: (672, 9067)
Ненулевых элементов в матрице: 100004
Доля ненулевых элементов: 0.01641286822438251
Среднее значение ненулевых элементов: 1.0
Максимальное значение ненулевых элементов: 1
Минимальное значение ненулевых элементов: 1


In [15]:
from scipy.spatial.distance import pdist, squareform

# pdist умеет всё "из коробки", но только с плотными матрицами
# плотная матрица может не умещаться в память - это нормально
similarity_matrix = squareform(
    1 - pdist(
        user_item_matrix.todense().T,
        metric="cosine"
    )
)
print(similarity_matrix.shape)

(9067, 9067)


In [12]:
print(movies[movies["title"].str.contains("Harry Potter")][["movieId", "title"]])
ratings[ratings["movieId"] == 4896].head(1)

      movieId                                              title
3812     4896  Harry Potter and the Sorcerer's Stone (a.k.a. ...
4325     5816     Harry Potter and the Chamber of Secrets (2002)
5399     8368    Harry Potter and the Prisoner of Azkaban (2004)
6294    40815         Harry Potter and the Goblet of Fire (2005)
6735    54001   Harry Potter and the Order of the Phoenix (2007)
7274    69844      Harry Potter and the Half-Blood Prince (2009)
7670    81834  Harry Potter and the Deathly Hallows: Part 1 (...
7842    88125  Harry Potter and the Deathly Hallows: Part 2 (...


,userId,movieId,rating,timestamp,movie_id
661,8,4896,3.5,1154400398,3811


In [13]:
# представление фильма в виде булевого вектора
movie_id = 3811
print(len(user_item_matrix[:, movie_id].data))
print(user_item_matrix[:, movie_id])

105
  (8, 0)	1
  (15, 0)	1
  (17, 0)	1
  (22, 0)	1
  (23, 0)	1
  (30, 0)	1
  (52, 0)	1
  (56, 0)	1
  (61, 0)	1
  (69, 0)	1
  (73, 0)	1
  (77, 0)	1
  (78, 0)	1
  (84, 0)	1
  (88, 0)	1
  (89, 0)	1
  (103, 0)	1
  (105, 0)	1
  (128, 0)	1
  (130, 0)	1
  (133, 0)	1
  (134, 0)	1
  (136, 0)	1
  (140, 0)	1
  (148, 0)	1
  :	:
  (523, 0)	1
  (524, 0)	1
  (525, 0)	1
  (531, 0)	1
  (542, 0)	1
  (553, 0)	1
  (561, 0)	1
  (562, 0)	1
  (570, 0)	1
  (575, 0)	1
  (577, 0)	1
  (580, 0)	1
  (598, 0)	1
  (599, 0)	1
  (607, 0)	1
  (612, 0)	1
  (615, 0)	1
  (620, 0)	1
  (623, 0)	1
  (624, 0)	1
  (627, 0)	1
  (648, 0)	1
  (654, 0)	1
  (665, 0)	1
  (671, 0)	1


In [16]:
from scipy.spatial import distance

def get_similar_movies(movie_id: int, n: int) -> list:
    return (-similarity_matrix[movie_id, :]).argsort()[:n]

similar_movies = get_similar_movies(3811, 5)
movies[movies["movieId"].isin(
    ratings[ratings["movie_id"].isin(similar_movies)]["movieId"].tolist()
)]

,movieId,title,genres
3847,4963,Ocean's Eleven (2001),Crime|Thriller
4325,5816,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
4693,6539,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
5399,8368,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy|IMAX
6294,40815,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller|IMAX
